# Connecting to your Emails vis IMAP (Python)

In this tutorial, we will be looking at how to connect to your emails via imap!

This video is based on the medium article below:
https://medium.com/@amnahhmohammed/natural-language-processing-for-emails-9c1cf5f74f48



#### Step #1: Import libraries and necessary packages

In [ ]:
import imaplib
import email
import regex as re
import codecs
import string


#### Step #2: “Connecting” to our email.

In [ ]:
# create an IMAP4_SSL class instance
imap = imaplib.IMAP4_SSL("imap.gmail.com")

# login with your Gmail account credentials
username = "email@gmail.com"
password = "APP PASSWORD" #should be an app password, to create one follow this link:https://support.google.com/accounts/answer/185833
imap.login(username, password)



('OK', [b'amnahslab@gmail.com authenticated (Success)'])

#### Step #3: Load Emails

In [ ]:
for i in imap.list()[1]:
    l = i.decode().split(' "/" ')
    print(l[0] + " = " + l[1])


(\HasNoChildren) = "INBOX"
(\HasChildren \Noselect) = "[Gmail]"
(\All \HasNoChildren) = "[Gmail]/All Mail"
(\HasNoChildren \Trash) = "[Gmail]/Bin"
(\Drafts \HasNoChildren) = "[Gmail]/Drafts"
(\HasNoChildren \Important) = "[Gmail]/Important"
(\HasNoChildren \Sent) = "[Gmail]/Sent Mail"
(\HasNoChildren \Junk) = "[Gmail]/Spam"
(\Flagged \HasNoChildren) = "[Gmail]/Starred"


In [ ]:
imap.select('"INBOX"')
status, messages = imap.search(None, 'UNSEEN')

# iterate over the messages and retrieve their contents
for num in messages[0].split()[::-1]:
    _, msg = imap.fetch(num, "(RFC822)")
    message = email.message_from_bytes(msg[0][1])
    if message.is_multipart():
      for part in message.walk():
        if part.get_content_type() == "text/plain":
            content = part.get_payload(decode=True)
            body=content.decode("utf-8")
            break
        else:
        # if no text/plain part was found, use the first part as the body
          content = message.get_payload(0).get_payload(decode=True)
          body=content.decode("utf-8")
    else:
    # if the message is not multipart, just use the message body
      content = message.get_payload(decode=True)
      body=content.decode("utf-8")

    # print the message details
    subject_header = message['Subject']
    decoded_subject = email.header.decode_header(subject_header)
    subject = decoded_subject[0][0]
    print("Subject:", subject)
    print("From:", message["From"])
    print("Date:", message["Date"])
    print("Body", body)


#### Step #4: NLP Task (Next Video!)

In [ ]:
from transformers import pipeline, AutoTokenizer
summarizer = pipeline("summarization", model="knkarthick/MEETING_SUMMARY")
text = '''The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct.
'''
summarizer(text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/337 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

[{'summary_text': 'The Eiffel Tower was built in 1889. It is the tallest structure in Paris and the second tallest free-standing structure in France after the Millau Viaduct. In 1930, it surpassed the Washington Monument to become the tallest man-made structure in the world. It was'}]

In [ ]:
summarizer = pipeline("summarization", model="knkarthick/MEETING_SUMMARY", tokenizer=AutoTokenizer.from_pretrained("knkarthick/MEETING_SUMMARY"))

In [ ]:
imap.select('"INBOX"')
status, messages = imap.search(None, 'UNSEEN')
summaryofsentemails=[]

# iterate over the messages and retrieve their contents
for num in messages[0].split()[::-1]:
    _, msg = imap.fetch(num, "(RFC822)")
    message = email.message_from_bytes(msg[0][1])
    if message.is_multipart():
      for part in message.walk():
        if part.get_content_type() == "text/plain":
            content = part.get_payload(decode=True)
            body=content.decode("utf-8")
            break
        else:
        # if no text/plain part was found, use the first part as the body
          content = message.get_payload(0).get_payload(decode=True)
          body=content.decode("utf-8")
    else:
    # if the message is not multipart, just use the message body
      content = message.get_payload(decode=True)
      body=content.decode("utf-8")


        # print the message details
    subject_header = message['Subject']
    decoded_subject = email.header.decode_header(subject_header)
    subject = decoded_subject[0][0]
    print("Subject:", subject)
    print("From:", message["From"])
    print("Date:", message["Date"])
    # Generate the summary
    content_summarised = summarizer(body, max_length=100,min_length=10, truncation=True)[0]['summary_text']
    print(content_summarised)
    summary={"Subject": subject,"From":str(message["From"]), "Date": str(message["Date"]),
      "Email ":content_summarised}
    summaryofsentemails.append(summary)



In [ ]:
summaryofsentemails